In [1]:
#  Install Required Libraries
!pip install faiss-cpu sentence-transformers transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:

# 📚 Imports
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [3]:
# 🧠 Load Embedding Model
retriever_model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# 📥 Load Product Data (Upload CSV)
from google.colab import files
uploaded = files.upload()
df = pd.read_csv(next(iter(uploaded)))


Saving Laptop.csv to Laptop.csv


In [5]:
# 🧹 Prepare Data
df.fillna('', inplace=True)
df['name'] = df['Brand'].astype(str) + " " + df['Model'].astype(str) + " " + df['Series'].astype(str)
df['description'] = (
    "Processor: " + df['Processor'].astype(str) + " (" + df['Processor_Gen'].astype(str) + "), " +
    "RAM: " + df['RAM'].astype(str) + "GB, " +
    "HDD: " + df['Hard_Disk_Capacity'].astype(str) + ", " +
    "OS: " + df['OS'].astype(str)
)
df['price'] = df['Price']
df['ram'] = df['RAM']
products = df.to_dict(orient='records')


<ipython-input-5-5d6112722e1f>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


In [6]:
# 🔍 Embed Product Descriptions
product_texts = [f"{p['name']} {p['description']}" for p in products]
product_embeddings = retriever_model.encode(product_texts, convert_to_numpy=True)


In [7]:
# 🧠 Create FAISS Index
embedding_dim = product_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(product_embeddings)


In [8]:
# 🔎 Define Product Search + Prompt Generator
def genai_product_search(query, max_price=None, min_ram=None, top_k=5):
    query_embedding = retriever_model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k * 2)

    retrieved = []
    for i in indices[0]:
        p = products[i]
        if (max_price is None or p.get("price", 1e9) <= max_price) and \
           (min_ram is None or p.get("ram", 0) >= min_ram):
            retrieved.append(p)
        if len(retrieved) >= top_k:
            break

    if not retrieved:
        return "❌ No products matched your criteria.", ""

    context = "\n".join([
        f"{i+1}. {p['name']} - ₹{p['price']} - {p['ram']}GB RAM\n{p['description'][:300]}..."
        for i, p in enumerate(retrieved)
    ])

    prompt = f"""You are a helpful AI assistant.

User is looking for a laptop. Based on the options below, recommend the **best one** with a short explanation.

Requirements:
- Price: Under ₹{max_price}
- RAM: At least {min_ram}GB
- Battery life: Prefer longer
- Usage: Remote work

Laptops available:
{context}

💡 Recommendation:"""
    return prompt, context


In [9]:
# 🤖 Load Free Open-Source LLM
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:

# 🧠 Text Generator Function
def generate_response(prompt, max_new_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [11]:
# 🔁 Full Flow: Search + Prompt + Response
query = "Need a laptop under ₹50000 with good RAM and battery backup for remote work"
prompt, context = genai_product_search(query, max_price=50000, min_ram=8, top_k=5)

print("📄 Prompt Sent to LLM:\n", prompt)

print("\n🤖 AI Recommendation:")
print(generate_response(prompt))

📄 Prompt Sent to LLM:
 You are a helpful AI assistant.

User is looking for a laptop. Based on the options below, recommend the **best one** with a short explanation.

Requirements:
- Price: Under ₹50000
- RAM: At least 8GB
- Battery life: Prefer longer
- Usage: Remote work

Laptops available:
1. Lenovo IdeaPad 3 - ₹49940 - 8.0GB RAM
Processor: 5500U (-), RAM: 8.0GB, HDD: 512 GB SSD, OS: Windows 11 Home...

💡 Recommendation:

🤖 AI Recommendation:
Lenovo IdeaPad 3
